In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install torch transformers sentencepiece h5py

In [ ]:
!wget -nc -P protT5/sec_struct_checkpoint http://data.bioembeddings.com/public/embeddings/feature_models/t5/secstruct_checkpoint.pt

In [ ]:
# whether to retrieve embeddings for each residue in a protein
# --> Lx1024 matrix per protein with L being the protein's length
# as a rule of thumb: 1k proteins require around 1GB RAM/disk
per_residue = True
per_residue_path = "./protT5/output/per_residue_embeddings.h5" # where to store the embeddings

# whether to retrieve per-protein embeddings
# --> only one 1024-d vector per protein, irrespective of its length
per_protein = True
per_protein_path = "./protT5/output/per_protein_embeddings.h5" # where to store the embeddings

# make sure that either per-residue or per-protein embeddings are stored
assert per_protein is True or per_residue is True or sec_struct is True, print(
    "Minimally, you need to active per_residue, per_protein or sec_struct. (or any combination)")

In [ ]:
def load_sec_struct_model():
  checkpoint_dir="./protT5/sec_struct_checkpoint/secstruct_checkpoint.pt"
  state = torch.load( checkpoint_dir )
  model = ConvNet()
  model.load_state_dict(state['state_dict'])
  model = model.eval()
  model = model.to(device)
  print('Loaded sec. struct. model from epoch: {:.1f}'.format(state['epoch']))

  return model

In [ ]:


# Generate embeddings via batch-processing
# per_residue indicates that embeddings for each residue in a protein should be returned.
# per_protein indicates that embeddings for a whole protein should be returned (average-pooling)
# max_residues gives the upper limit of residues within one batch
# max_seq_len gives the upper sequences length for applying batch-processing
# max_batch gives the upper number of sequences per batch
def get_embeddings( model, tokenizer, seqs, per_residue, per_protein, sec_struct,
                   max_residues=4000, max_seq_len=1000, max_batch=100 ):

    if sec_struct:
      sec_struct_model = load_sec_struct_model()

    results = {"residue_embs" : dict(),
               "protein_embs" : dict(),
               "sec_structs" : dict()
               }

    # sort sequences according to length (reduces unnecessary padding --> speeds up embedding)
    seq_dict   = sorted( seqs.items(), key=lambda kv: len( seqs[kv[0]] ), reverse=True )
    start = time.time()
    batch = list()
    for seq_idx, (pdb_id, seq) in enumerate(seq_dict,1):
        seq = seq
        seq_len = len(seq)
        seq = ' '.join(list(seq))
        batch.append((pdb_id,seq,seq_len))

        # count residues in current batch and add the last sequence length to
        # avoid that batches with (n_res_batch > max_residues) get processed
        n_res_batch = sum([ s_len for  _, _, s_len in batch ]) + seq_len
        if len(batch) >= max_batch or n_res_batch>=max_residues or seq_idx==len(seq_dict) or seq_len>max_seq_len:
            pdb_ids, seqs, seq_lens = zip(*batch)
            batch = list()

            # add_special_tokens adds extra token at the end of each sequence
            token_encoding = tokenizer.batch_encode_plus(seqs, add_special_tokens=True, padding="longest")
            input_ids      = torch.tensor(token_encoding['input_ids']).to(device)
            attention_mask = torch.tensor(token_encoding['attention_mask']).to(device)

            try:
                with torch.no_grad():
                    # returns: ( batch-size x max_seq_len_in_minibatch x embedding_dim )
                    embedding_repr = model(input_ids, attention_mask=attention_mask)
            except RuntimeError:
                print("RuntimeError during embedding for {} (L={})".format(pdb_id, seq_len))
                continue

            if sec_struct: # in case you want to predict secondary structure from embeddings
              d3_Yhat, d8_Yhat, diso_Yhat = sec_struct_model(embedding_repr.last_hidden_state)


            for batch_idx, identifier in enumerate(pdb_ids): # for each protein in the current mini-batch
                s_len = seq_lens[batch_idx]
                # slice off padding --> batch-size x seq_len x embedding_dim
                emb = embedding_repr.last_hidden_state[batch_idx,:s_len]
                if sec_struct: # get classification results
                    results["sec_structs"][identifier] = torch.max( d3_Yhat[batch_idx,:s_len], dim=1 )[1].detach().cpu().numpy().squeeze()
                if per_residue: # store per-residue embeddings (Lx1024)
                    results["residue_embs"][ identifier ] = emb.detach().cpu().numpy().squeeze()
                if per_protein: # apply average-pooling to derive per-protein embeddings (1024-d)
                    protein_emb = emb.mean(dim=0)
                    results["protein_embs"][identifier] = protein_emb.detach().cpu().numpy().squeeze()


    passed_time=time.time()-start
    avg_time = passed_time/len(results["residue_embs"]) if per_residue else passed_time/len(results["protein_embs"])
    print('\n############# EMBEDDING STATS #############')
    print('Total number of per-residue embeddings: {}'.format(len(results["residue_embs"])))
    print('Total number of per-protein embeddings: {}'.format(len(results["protein_embs"])))
    print("Time for generating embeddings: {:.1f}[m] ({:.3f}[s/protein])".format(
        passed_time/60, avg_time ))
    print('\n############# END #############')
    return results

In [ ]:
from transformers import T5EncoderModel, T5Tokenizer

import h5py
import time


import os
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import re
from collections import defaultdict
import random

seed = 7

torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F

from transformers import Trainer, TrainingArguments, EvalPrediction
from datasets import load_dataset

# from seqeval.metrics import accuracy_score, f1_score, precision_score, recall_score
from scipy import stats
from functools import partial
import pandas as pd
from tqdm.auto import tqdm

# Load Model and Data

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Available device:', device)
def check_len(some_list):
    for i in some_list:
        print(len(i))


def get_num_params(model):
    return sum(p.numel() for p in model.parameters())

def get_T5_model():
    model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_half_uniref50-enc")
    model = model.to(device) # move model to GPU
    model = model.eval() # set model to evaluation model
    tokenizer = T5Tokenizer.from_pretrained('Rostlab/prot_t5_xl_half_uniref50-enc', do_lower_case=False)

    return model, tokenizer

In [ ]:
model, tokenizer = get_T5_model()
model.eval()
model.to(device=device)
print(f"Number of parameters:", get_num_params(model))


In [ ]:
def extract_sequence_id(index_name):
    """
    Extract sequence ID from index like 'sequence_0_window_1'
    
    Parameters:
    -----------
    index_name : str
        Index name in format 'sequence_[id]_window_[window_id]'
    
    Returns:
    --------
    str
        The sequence ID (e.g., '0' from 'sequence_0_window_1')
    """
    # Use regex to extract sequence ID
    match = re.search(r'sequence_(\d+)_window_\d+', index_name)
    if match:
        return match.group(1)
    else:
        # Alternative pattern matching if format is different
        parts = index_name.split('_')
        if len(parts) >= 4 and parts[0] == 'sequence':
            return parts[1]
    return None

def sequence_based_split(df, val_size=0.2, random_state=42):
    """
    Split dataframe based on sequence IDs to prevent data leakage.
    All windows from the same sequence will be in the same split.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame with index containing sequence and window information
    test_size : float
        Proportion of sequences for test set
    val_size : float
        Proportion of remaining sequences for validation set
    random_state : int
        Random seed for reproducibility
    
    Returns:
    --------
    tuple
        (train_df, val_df, test_df, split_info)
    """
    # Extract sequence IDs from index
    sequence_ids = []
    for idx in df['id']:
        seq_id = extract_sequence_id(str.lower(idx))
        if seq_id is not None:
            sequence_ids.append(seq_id)
        else:
            raise ValueError(f"Could not extract sequence ID from index: {idx}")
    
    # Add sequence IDs to dataframe (temporary)
    df_temp = df.copy()
    df_temp['sequence_id'] = sequence_ids
    
    # Get unique sequence IDs
    unique_sequences = list(set(sequence_ids))
    print(f"Total unique sequences: {len(unique_sequences)}")
    
    # Count windows per sequence
    sequence_counts = defaultdict(int)
    for seq_id in sequence_ids:
        sequence_counts[seq_id] += 1
    
    print(f"Windows per sequence: min={min(sequence_counts.values())}, "
          f"max={max(sequence_counts.values())}, "
          f"mean={np.mean(list(sequence_counts.values())):.1f}")
    
    # First split: separate test sequences
    train_sequences, val_sequences = train_test_split(
        unique_sequences, 
        test_size=val_size, 
        random_state=random_state
    )
    
    # Create splits based on sequence membership
    train_df = df_temp[df_temp['sequence_id'].isin(train_sequences)].drop('sequence_id', axis=1)
    val_df = df_temp[df_temp['sequence_id'].isin(val_sequences)].drop('sequence_id', axis=1)
    
    # Create split information
    split_info = {
        'train_sequences': sorted(train_sequences),
        'val_sequences': sorted(val_sequences),
        'train_windows': len(train_df),
        'val_windows': len(val_df),
        'total_windows': len(df)
    }
    
    print(f"Split results:")
    print(f"  Train: {len(train_sequences)} sequences, {len(train_df)} windows")
    print(f"  Val:   {len(val_sequences)} sequences, {len(val_df)} windows") 
    
    return train_df, val_df, split_info

def verify_no_leakage(train_df, val_df):
    """
    Verify that there's no sequence leakage between splits.
    
    Parameters:
    -----------
    train_df, val_df, test_df : pandas.DataFrame
        The split dataframes
    
    Returns:
    --------
    bool
        True if no leakage detected, False otherwise
    """
    # Extract sequence IDs from each split
    train_seqs = set([extract_sequence_id(str(idx)) for idx in train_df.index])
    val_seqs = set([extract_sequence_id(str(idx)) for idx in val_df.index])
    
    # Check for overlaps
    train_val_overlap = list(train_seqs.intersection(val_seqs))
    
    if len(train_val_overlap) > 1 or train_val_overlap[0] is not None:
        print("❌ DATA LEAKAGE DETECTED!")
        if train_val_overlap:
            print(f"  Train-Val overlap: {train_val_overlap}")
        return False
    else:
        print("✅ No data leakage detected. All sequences are properly separated.")
        return True

# Example usage and demonstration
def split_dataset_process(df):
    """Demonstrate the sequence-based splitting functionality"""
    
    # Test regular sequence-based split
    print("=== Regular Sequence-Based Split Process ===")
    train_df, val_df, split_info = sequence_based_split(
        df, val_size=0.25, random_state=42
    )
    
    print()
    verify_no_leakage(train_df, val_df)
    print()

    
    return train_df, val_df, split_info



In [ ]:
train_df =  pd.read_csv('/kaggle/input/preprocess-f/Train_PS4_T5.csv')
train_df['len_seq_aa'] = train_df['aa_sequence'].apply(lambda x : len(str(x)))
train_df['len_ssp8'] = train_df['ssp_sequence'].apply(lambda x : len(str(x)))
print("=== Training PS4 Dataset ===")
print(train_df.info())
val_df =  pd.read_csv('/kaggle/input/preprocess-f/Val_PS4_T5.csv')
val_df['len_seq_aa'] = val_df['aa_sequence'].apply(lambda x : len(str(x)))
val_df['len_ssp8'] = val_df['ssp_sequence'].apply(lambda x : len(str(x)))
print("=== Validation PS4 Dataset ===")
print(val_df.info())

df_cb433 = pd.read_csv('/kaggle/input/preprocess-f/CB433_T5.csv')
# df_cb433 = pd.read_csv('/kaggle/input/preprocessed-v4/CB433_108cut.csv')
print("=== Test CB433 Dataset ===")
print(df_cb433.info())
df_cb433['len_seq_aa'] = df_cb433['aa_sequence'].apply(lambda x : len(str(x)))
df_cb433['len_ssp8'] = df_cb433['ssp_sequence'].apply(lambda x : len(str(x)))
print(df_cb433.describe())

In [ ]:
# val_df = pd.read_csv('/kaggle/input/preprocessed-v4/Val_PS4.csv')
dfs = [
train_df,
       val_df,
df_cb433]
# dfs = [val_df, df_cb433]

# train_df.to_csv('Train_PS4.csv')
# val_df.to_csv('Val_PS4.csv')

# Processing Dataset

In [ ]:
def tokenize_sequence(sequence: str):
        # Define the special tokens
        # special_tokens = {"<cls>", "<eos>", "<pad>"}
        special_tokens = ["</s>", "<pad>"]
        # special_tokens = {"[CLS]", "[PAD]"}
        result = []
        i = 0
        n = len(sequence)
        limit = 108
        while i < n:
            # Check if the current position starts a special token
            if sequence[i:i+4] in special_tokens[0]:  # Handle </s>
                result.append(sequence[i:i+4])
                i += 4  # Skip the length of the special token
            elif sequence[i:i+5] in special_tokens[1]:  # Handle <pad>
                result.append(sequence[i:i+5])
                i += 5  # Skip the length of the special token
            else:
                # Add individual characters
                result.append(sequence[i])
                i += 1
        if len(result) < limit:
            num_pad = limit- len(result)
            result += ['<pad>']*num_pad
        if len(result)> limit:
            result = result[:limit]
    
        return result
    
def preprocessed(df_col):
    processed_sequences = []
    for seq in df_col:
        # prep_seq = list(seq)
        prep_seq = tokenize_sequence(seq)
        processed_sequences.append(prep_seq)
    return processed_sequences
    
def preprocess_dataset(sequences, labels, 
                       # disorder, 
                       max_length=None, st = True):
    
    if st:
        sequences = ["".join(str(seq).split()) for seq in sequences]
    else:
        sequences = preprocessed(sequences)
    
    if max_length is None:
        max_length = len(max(sequences, key=lambda x: len(x)))

    seqs = [list(seq)[:max_length] for seq in sequences]
    
    labels = ["".join(str(label).split()) for label in labels]
    labels = [list(label)[:max_length] for label in labels]
    
    # disorder = [" ".join(disorder.split()) for disorder in disorder]
    # disorder = [disorder.split()[:max_length] for disorder in disorder]
    
    assert len(seqs) == len(labels)
    return seqs, labels


def embed_dataset(model, sequences, 
                  # tokenizer, device, 
                  shift_left=0, shift_right=-1, max_length=108, st_status=True):
    """
    Embed sequences using a pre-trained model and adjust the attention mask to only include 
    actual content (set to 0 after the last non-padding token).
    
    Args:
        model: Pre-trained model for embedding the sequences
        sequences: List of sequences to embed
        tokenizer: Tokenizer to convert sequences to token IDs
        device: Device to run the model on (e.g., 'cuda' or 'cpu')
        shift_left: Number of tokens to exclude from the beginning
        shift_right: Number of tokens to exclude from the end (negative indexing)
        max_length: Maximum sequence length for padding/truncation
        st_status: Whether to add special tokens
        
    Returns:
        Tuple of (inputs_id, inputs_embedding, adjusted_attention_masks)
    """
    inputs_embedding = []
    inputs_id = []
    adjusted_attention_masks = []
    
    with torch.no_grad():
        for sample in tqdm(sequences):
            # Tokenize the input
            ids = tokenizer.batch_encode_plus(
                sample, 
                add_special_tokens=True, 
                # max_length=max_length,
                padding=True, 
                # is_split_into_words=True, 
                truncation=True, 
                return_tensors="pt"
            )
            
            # Create a new attention mask based on the actual content
            input_ids = torch.tensor(ids['input_ids']).to(device)
            
            # Find the padding token ID (usually 0 or 1 depending on the tokenizer)
            padding_token_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else 0
            
            # # Find the position of the last non-padding token
            # non_padding_positions = np.where(input_ids != padding_token_id)[0]
            
            # if len(non_padding_positions) > 0:
            #     # If there are non-padding tokens, find the last one
            #     last_token_pos = non_padding_positions[-1]
                
            #     # Create a new attention mask: 1s up to and including the last token, 0s after
            #     adjusted_mask = np.zeros(max_length, dtype=np.int64)
            #     adjusted_mask[:last_token_pos + 1] = 1
            # else:
            #     # If all tokens are padding tokens (unlikely but possible)
            #     adjusted_mask = np.zeros(max_length, dtype=np.int64)
            
            # Convert to tensor
            attention_mask = torch.tensor(ids['attention_mask']).to(device)
            
            # Generate embeddings
            embedding = model.encoder(input_ids, attention_mask=attention_mask)
            
            # # Apply shifts if needed
            # if shift_right == -1:
            #     embedding_np = embedding[0].detach().cpu().numpy()[shift_left:]
            # else:
            #     embedding_np = embedding[0].detach().cpu().numpy()[shift_left:shift_right]
            
            inputs_embedding.append(embedding.last_hidden_state.to(device))
            # inputs_id.append(ids)
            # adjusted_attention_masks.append(adjusted_mask)
    
    return inputs_embedding


# Processing Labels

In [ ]:
# Consider each label as a tag for each token
# training_sequences, training_labels = preprocess_dataset(df_pad['aa_sequence'], 
#                                                         df_pad['ssp_sequence'], 
#                                                          st = False
#                                                         )
# unique_tags = set(tag for doc in training_labels for tag in doc)
# unique_tags.remove('X')
# tag2id = {tag: id for id, tag in enumerate(unique_tags)}
# id2tag = {id: tag for tag, id in tag2id.items()}
# del training_sequences, training_labels
# print(tag2id)

{'T': 0, 'S': 1, 'G': 2, 'P': 3, 'L': 4, 'E': 5, 'B': 6, 'H': 7, 'I': 8}

In [ ]:

def encode_tags(labels, max_length=108, padding_value = -100):
    """
    Encode tags to IDs and pad to a fixed length.
    
    Args:
        labels: List of lists, where each inner list contains tags for a document
        max_length: Maximum length for padding (default: 108)
    
    Returns:
        List of lists with tag IDs and padding
    """
    encoded_labels = []
    
    for doc in labels:
        # Convert tags to IDs, skip unknown tags
        doc_ids = [tag2id.get(tag, padding_value) for tag in doc]  # Use -100 for unknown tags
        
        # Truncate if longer than max_length
        if len(doc_ids) > max_length:
            doc_ids = doc_ids[:max_length]
        # Add padding if shorter than max_length
        else: 
            padding = [padding_value] * (max_length - len(doc_ids))
            doc_ids = doc_ids + padding
            
        encoded_labels.append(doc_ids)
    
    return encoded_labels

In [ ]:

def tensor_to_numpy(tensor):
    """Safely convert tensor to numpy array"""
    # Check if it's a PyTorch tensor
    if torch.is_tensor(tensor):
        return tensor.detach().cpu().numpy()
    elif isinstance(tensor, np.ndarray):
        return tensor
    elif isinstance(tensor, list):
        # Handle list of tensors
        converted_list = []
        for item in tensor:
            if torch.is_tensor(item):
                converted_list.append(item.detach().cpu().numpy())
            else:
                converted_list.append(item)
        return np.array(converted_list)
    else:
        # For other types, convert to numpy array safely
        try:
            return np.array(tensor)
        except Exception as e:
            print(f"Error converting {type(tensor)} to numpy: {e}")
            raise
# processing to numpy special because it got error terus menerus naon gw gatau
def convert_embeddings_to_numpy(training_embeddings):
    """Convert list of same-shaped tensors to numpy array"""
    # Convert each tensor individually first
    numpy_embeddings = []
    for i, embedding in enumerate(training_embeddings):
        if torch.is_tensor(embedding):
            numpy_emb = embedding.detach().cpu().numpy()
            # print(f"Converted tensor {i}: shape {numpy_emb.shape}, dtype {numpy_emb.dtype}")
            numpy_embeddings.append(numpy_emb)
        else:
            print(f"Item {i} is not a tensor: {type(embedding)}")
            numpy_embeddings.append(embedding)
    
    # Stack them into a single array
    try:
        stacked = np.stack(numpy_embeddings, axis=0)
        # print(f"Final stacked shape: {stacked.shape}")
        return stacked
    except Exception as e:
        # print(f"Error stacking: {e}")
        return numpy_embeddings
        
def memory_efficient_processing(model, df, tag2id, label, chunk_size=1000):
    total_chunks = (len(df) - 1) // chunk_size + 1
    results = []
    
    with tqdm(total=total_chunks, desc="Processing chunks") as pbar:
        input_column_name = 'aa_sequence'
        labels_column_name = 'ssp_sequence'
        padding_value = -100
        # Create output directory
        output_dir = './embedding_T5_result/'
        os.makedirs(output_dir, exist_ok=True)

        for i in range(0, len(df), chunk_size):
            # Get chunk
            chunk_num = i // chunk_size
            chunk = df.iloc[i:i + chunk_size].copy()
            print(f"Processing chunk {chunk_num + 1}/{(len(df)-1)//chunk_size + 1}")

            #preprocessing dataset
            training_sequences = chunk[input_column_name]
            training_labels = chunk[labels_column_name]

            training_sequences, training_labels = preprocess_dataset(training_sequences, 
                                                        training_labels, 
                                                         st = False
                                                        )
             # Clear memory
            del chunk
            # Process embeddings
            training_embeddings = embed_dataset(model, 
                                                training_sequences,
                                                st_status = True)
             # Clear memory
            del training_sequences
            # Save to NPZ
            npz_path = os.path.join(output_dir, f'{label}_sequence_chunk_{chunk_num:04d}.npz')
            
            
            # Save embeddings and any metadata
            np.savez_compressed(
                npz_path,
                embeddings=convert_embeddings_to_numpy(training_embeddings),
                # indices=chunk.index.values,  # Original indices
                chunk_start=i,
                chunk_end=min(i + chunk_size, len(df))
            )
            del training_embeddings
            
            #process label encoding
            train_labels_encodings = torch.tensor(encode_tags(training_labels, padding_value = padding_value))
            train_tensor_enc = F.one_hot(train_labels_encodings.clamp(min=0), num_classes=9)
            # Replace one-hot vectors for padding values with all zeros
            # This creates a mask where True indicates padding positions
            padding_mask = (train_labels_encodings == padding_value).unsqueeze(-1)
            # Zero out the one-hot vectors at padding positions
            train_tensor_enc = train_tensor_enc.masked_fill(padding_mask, 0)
            print(f"Final shape: {train_tensor_enc.shape}") 

            del training_labels

            # Save to NPZ
            npz_path = os.path.join(output_dir, f'{label}_labels_chunk_{chunk_num:04d}.npz')
            
            # Save embeddings and any metadata
            np.savez_compressed(
                npz_path,
                embeddings=tensor_to_numpy(train_tensor_enc),
                # indices=chunk.index.values,  # Original indices
                chunk_start=i,
                chunk_end=min(i + chunk_size, len(df))
            )
            # Clear memory
            del train_labels_encodings, train_tensor_enc, padding_mask
            
            # Update progress
            pbar.update(1)

In [ ]:
tag2id = {'S': 0, 'H': 1, 'B': 2, 'I': 3, 'L': 4, 'P': 5, 'G': 6, 'E': 7, 'T': 8}

In [ ]:
label_dfs = ['train', 'val', 'test']
# label_dfs = ['val']
for i in range(len(dfs)):
    memory_efficient_processing(model, dfs[i], tag2id, label = label_dfs[i], chunk_size=5000)